In [ ]:
import random
import utils
import numpy as np
import matplotlib.pyplot as plt

stack_size=10

# LeducGame class definition

In [ ]:
class LeducGame:
    deck = []
    actions = [0,1,2] #0 is fold, 1 is check, 2 push
    firstplayer=None; #0 if player1 and 1 if player2
    hand_player1=0;
    hand_player2=0;
    boardcard=0;
    result=0;
    step=0
    roundGame=0 #0 preflop, 1 postflop
    pot=None
    stack1=None
    stack2=None
    lastAction=None
    current_player=None
    game_is_over=None #0 game is not over, #1 game is over
    #inititate a game
    def __init__(self):
        self.deck = [0,0,1,1,2,2]
        
        #deal card to game from deck
        self.hand_player1=utils.choose_and_remove(self.deck)
        self.hand_player2=utils.choose_and_remove(self.deck)
        self.boardcard=utils.choose_and_remove(self.deck)
        self.result=self.get_result()
        self.firstplayer=random.randrange(0,2)
        self.step=0
        self.roundGame=0
        self.pot=0
        self.stack1=stack_size
        self.stack2=stack_size
        self.current_player=self.firstplayer
        self.game_is_over=0
        
    #allow to print leduc game state
    def __str__(self):
        return "Player1 = {} \nPlayer2 = {} \nBoard = {} \nDeck = {}\nResult = {}".format(self.hand_player1,self.hand_player2,self.boardcard,self.deck, self.result)
     
    def get_firstplayer(self):
            return self.firstplayer
        
    def get_hand_player1(self):
            return self.hand_player1
    
    def get_hand_player2(self):
            return self.hand_player2
        
    def get_boardcard(self):
            return self.boardcard
        
    def get_current_player(self):
            return self.current_player
    #result() : 
    # 0  -> draw
    # 1  -> player1 win
    #-1  -> player2 win
    def get_result(self): #determine the best hand
        #Pairs
        if (self.hand_player1==self.boardcard):
            result=1
        elif (self.hand_player2==self.boardcard):
            result=-1
        #Highest card
        elif (self.hand_player1>self.hand_player2):
            result=1
        elif(self.hand_player1<self.hand_player2):
            result=-1
        #Draw
        else:
            result=0
        return result    
    
    def allowed_actions(self, action):
        if (action==0):
            return None
        elif (action==1):
            return [0,1,2]
        elif (action==2):
            return [0, 2]
          
    
    def step(self, action):
        
        #STEP 1
        if(self.step==0):
            self.current_player=self.firsplayer
            self.pot=2
            self.stack1=self.stack1-1
            self.stack2=self.stack2-1
            if (action==0):
                self.game_is_over=1
                if(self.current_player==0):
                    gain=-1
                if(self.current_player==1):
                    gain=1                   
            
        #STEP 2
        elif(self.step==1):
            self.current_player=not(self.firsplayer)
            if (action==0):
                self.game_is_over=1
                if(self.current_player==0):
                    gain=-1
                if(self.current_player==1):
                    gain=1   
            elif(action==lastAction and action==1):
                self.roundGame=self.roundGame+1
                self.step=0
            elif(action==lastAction and action==2):
                self.game_is_over=1
                if(self.get_result()==0):
                    gain=self.pot/2
                if(self.get_result()==0):
                    gain=-self.pot/2
                
        #STEP 3
        elif(self.step==2):
            self.current_player=self.firsplayer
            if (action==0):
                self.game_is_over=1
                if(self.current_player==0):
                    gain=-1
                if(self.current_player==1):
                    gain=1   
            elif(action==lastAction and action==2):
                self.game_is_over=1
                if(self.get_result()==0):
                    gain=self.pot/2
                if(self.get_result()==0):
                    gain=-self.pot/2
            if (action==2):
                if(self.current_player==0):
                    self.pot=self.pot+self.stack1
                elif(self.current_player==0):
                    self.pot=self.pot+self.stack2
        
        if (action==2):
                if(self.current_player==0):
                    self.pot=self.pot+self.stack1
                    self.stack1=0
                elif(self.current_player==0):
                    self.pot=self.pot+self.stack2
                    self.stack2=0
                    
        self.step= self.step+1
        actions= allowed_actions(action)
        self.lastAction=action
        return gain,self.current_player, actions
        

# Environment

In [ ]:
class Environment:
    agent=None
    game=None
    opponent_action=None
    game_round=None #0 is round 1 and 1 is round 2 
    game_is_over=None #0 if false, 1 if true
    
    def __init__(self,agent):
        self.game=LeducGame()
        self.agent=agent
        self.agent.set_action(self.game.get_hand_player2())
        self.game_round=0
        
        self.game_is_over=0
        
        if (self.game.get_firstplayer()==1):
            self.opponent_action=self.agent.get_action()
        
        qAgentState= self.get_state()
        
        return qAgentState
        
        
    def __str__(self):
        return "{} \nOpponent_Action = {}\nStack = {}".format(self.game,self.agent.get_action(),self.stack)
    
    def get_state(self):
        hand1=self.game.get_hand_player1()
        first=self.game.get_firstplayer()
        boardcard= self.game.get_boardcard()
        
        if self.game_round == 0:
            if first is 0:
                return hand1
            elif(first==1 and self.opponent_action==1):
                return hand1+3
            elif(first==1 and self.opponent_action==2):
                return hand1+6
            
        elif self.game_round == 1:
            if first is 0:
                if hand1==0:
                    return boardcard+9
                elif hand1=1:
                    return boardcard+12
                elif hand1=2:
                    return boardcard+15
            elif (first==1 and self.opponent_action==1):
                if hand1==0:
                    return boardcard+18
                elif hand1=1:
                    return boardcard+21
                elif hand1=2:
                    return boardcard+24
            elif (first==1 and self.opponent_action==2):
                if hand1==0:
                    return boardcard+27
                elif hand1=1:
                    return boardcard+30
                elif hand1=2:
                    return boardcard+33
        
    def reward(self, real_winner):
        r=0
        
        if (game_is_over() == 1):
            if real_winner==0:
                r= self.pot/2
            else 
                r= -self.pot/2
        return r
    
    def reset(self):
        self.game=LeducGame() 
        self.agent.set_action(self.game.get_hand_player2())
        self.opponent_action=self.agent.get_action()
        
    def test(self,n):
        r=0
        for i in range(n):
            r=r+self.reward(self,qagent_action,state) 
            self.reset() 
        r=r/n
        return r
    
    def step(self, qagent_action):
        r=0
        state=None
        
        r,current_player,allowed_actions=self.game.step(qagent_action)
        self.agent.set_action(allowed_actions)
        self.opponent_action=self.agent.get_action()
        _,current_player,allowed_actions= self.game.step(self.opponent_action)
        state=self.get_state()
        
        return r, allowed_actions, state
            

## Random Agent

In [ ]:
class RandAgent():
    
    action=None
          
        
    def __init__(self):
        #qtable creation
        self.set_action(0)
        
    def set_action(self,allowed_actions):            
        self.action=random.choice(allowed_actions)
        
    def get_action(self):
        return self.action

## Random Agent

In [ ]:
class QAgent():
    qtable=[]
 # Agent class definition   state=None
    learning_rate=0.1
    
    state=None #0-5
    
    state_number=36
    
    perf=[]
    
    def __init__(self):
        #qtable creation
        self.qtable=np.zeros((self.state_number,3))
        self.perf=[]
        
    #allow to print leduc game state
    def __str__(self):
        return "State = {} \nQTable = {} \nLearning rate = {}".format(self.state,self.qtable,self.learning_rate)
    
    def explore_action(self, allowed_actions):
        action=random.choice(allowed_actions)
        return action
    
    def exploit_action(self, allowed_actions):
        action=utils.get_max_list(self.qtable[self.state], allowed_actions)
        return action
    
    def set_state(self, state):
        self.state=state
        
    def set_qtable(self,qtable):
        self.qtable=qtable
        
    def set_perf(self,perf):
        self.perf.append(perf)

    def update(self,action,reward):
        if(self.state!=-1):
            new_value = (1 - self.learning_rate) * self.qtable[self.state, action] +  self.learning_rate * reward
            self.qtable[self.state, action] = new_value